# Homework 3

```yaml
Course:   DS 5001 
Module:   03 Language Models
Topic:    Homework 3
Author:   Ryan Lipps
Date:     2/1/2024
```

In [1]:
import numpy as np
import pandas as pd
import textimporter

In [2]:
import configparser
config = configparser.ConfigParser()
config.read("../../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']

In [3]:
text_file = f"{data_home}/gutenberg/pg42324.txt"

In [4]:
ohco_pats = [('chap', r'^(?:LETTER|CHAPTER|PREFACE)\b', 'm')]
clip_pats = [r'START', r'END']
timporter = textimporter.TextImporter(src_file=text_file, ohco_pats=ohco_pats, clip_pats=clip_pats)
timporter.import_source().parse_tokens(special_tokens=['_'])
print(timporter.TOKENS.head())
print(timporter.gather_tokens(1))

Importing  /Users/ryanlipps/Documents/MSDS/DS5001/data/gutenberg/pg42324.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^(?:LETTER|CHAPTER|PREFACE)\b
Parsing OHCO level 1 para_num by delimiter \n\n
Parsing OHCO level 2 sent_num by delimiter [.?!;:]+
Parsing OHCO level 3 token_num by delimiter [\s',-,_,--]+
                                    token_str term_str
chap_id para_num sent_num token_num                   
1       0        0        0               The      the
                          1             event    event
                          2                on       on
                          3             which    which
                          4              this     this
                                                       para_num_str
chap_id para_num                                                   
1       0         the event on which this fiction is founded has...
        1         i have thus endeavoured to preserve the truth ...
        2         the 

/Users/ryanlipps/miniconda3/envs/ds-6001/lib/python3.10/site-packages/pandas/core/strings/object_array.py:343: FutureWarning: Possible set difference at position 9
  new_pat = re.compile(pat)


In [5]:
TOKENS = timporter.TOKENS
TOKENS.head()

token_str term_str
chap_id para_num sent_num token_num                   
1       0        0        0               The      the
                          1             event    event
                          2                on       on
                          3             which    which
                          4              this     this

In [6]:
VOCAB = timporter.extract_vocab().VOCAB
VOCAB

,n,n_chars,p,s,i,h
term_str,,,,,,
the,4248,3,0.055695,17.955038,4.166317,0.232042
and,2991,3,0.039214,25.500836,4.672473,0.183228
i,2858,1,0.037471,26.687544,4.738095,0.177540
of,2683,2,0.035176,28.428252,4.829253,0.169875
to,2118,2,0.027769,36.011804,5.170398,0.143575
...,...,...,...,...,...,...
execrated,1,9,0.000013,76273.000000,16.218885,0.000213
spectators,1,10,0.000013,76273.000000,16.218885,0.000213
constrained,1,11,0.000013,76273.000000,16.218885,0.000213


In [7]:
def get_ngrams(TOKEN, n=2, sent_key='sent_num'):

    OHCO = TOKEN.index.names
    grouper = list(OHCO)[:OHCO.index(sent_key)+1]

    PADDED = TOKEN.groupby(grouper)\
        .apply(lambda x: '<s> ' + ' '.join(x.term_str) + ' <s>')\
        .apply(lambda x: pd.Series(x.split()))\
        .stack().to_frame('term_str')
    PADDED.index.names = grouper + ['token_num']

    for i in range(1, n):
        PADDED = PADDED.join(PADDED.term_str.shift(-i), rsuffix=i)

    PADDED.columns = [f'w{j}' for j in range(n)]

    return PADDED

In [8]:
unigrams = get_ngrams(TOKENS, 1)
unigrams.head()

w0
chap_id para_num sent_num token_num       
1       0        0        0            <s>
                          1            the
                          2          event
                          3             on
                          4          which

In [9]:
bigrams = get_ngrams(TOKENS, 2)
bigrams.head()

w0     w1
chap_id para_num sent_num token_num              
1       0        0        0            <s>    the
                          1            the  event
                          2          event     on
                          3             on  which
                          4          which   this

In [10]:
trigrams = get_ngrams(TOKENS, 3)
trigrams.head()

w0     w1       w2
chap_id para_num sent_num token_num                       
1       0        0        0            <s>    the    event
                          1            the  event       on
                          2          event     on    which
                          3             on  which     this
                          4          which   this  fiction

## Question 1:
List six words that precede the word "monster," excluding stop words (and sentence boundary markers).

### Answer 1:

In [11]:
stopwords = [
    '<s>',
    'a',
    'an',
    'and',
    'are',
    'as',
    'at',
    'be',
    'but',
    'by',
    'for',
    'if',
    'in',
    'into',
    'is',
    'it',
    'no',
    'not',
    'of',
    'on',
    'or',
    'such',
    'that',
    'the',
    'their',
    'then',
    'there',
    'these',
    'they',
    'this',
    'to',
    'was',
    'will',
    'with'
]

trigrams.query('(w1 == "monster" or w2 == "monster")\
             and ~((w0.isin(@stopwords) or w1.isin(@stopwords)))')

w0       w1     w2
chap_id para_num sent_num token_num                            
12      3        17       25          miserable  monster   whom
17      8        0        1            abhorred  monster    <s>
22      25       4        23         detestable  monster    <s>
23      28       0        1             hideous  monster    <s>
31      4        9        5             hellish  monster  drink
        17       6        2            gigantic  monster   they

In [12]:
def get_ngram_counts(NGRAM):
    "Compress the sequences into counts"
    
    n = len(NGRAM.columns)
    C = [None for i in range(n)]
    
    for i in range(n):

        # Count distinct ngrams
        C[i] = NGRAM.iloc[:, :i+1].value_counts().to_frame('n').sort_index()
    
        # Get joint probabilities (MLE)
        C[i]['p'] = C[i].n / C[i].n.sum()
        C[i]['i'] = np.log2(1/C[i].p)

        # Get conditional probabilities (MLE)
        if i > 0:
            C[i]['cp'] = C[i].n / C[i-1].n
            C[i]['ci'] = np.log2(1/C[i].cp)
            
    return C

In [13]:
tgcounts = get_ngram_counts(trigrams)
tgcounts[2].sort_values('n')

n         p          i        cp        ci
w0     w1    w2                                                
life   which from    1  0.000012  16.396337  0.333333  1.584963
paper  it    is      1  0.000012  16.396337  1.000000  0.000000
       signs for     1  0.000012  16.396337  1.000000  0.000000
papers <s>   <s>     1  0.000012  16.396337  1.000000  0.000000
       can   come    1  0.000012  16.396337  1.000000  0.000000
...                ...       ...        ...       ...       ...
<s>    <s>   <s>   366  0.004243   7.880637  0.068411  3.869623
             the   373  0.004324   7.853305  0.069720  3.842291
             and   421  0.004881   7.678661  0.078692  3.667647
             but   457  0.005298   7.560287  0.085421  3.549273
             i     823  0.009541   6.711589  0.153832  2.700575

[67718 rows x 5 columns]

## Question 2:
List the following sentences in ascending order of bigram perplexity according to the language model generated from the text: 

The monster is on the ice.
Flowers are happy things.
I have never seen the aurora borealis.
He never knew the love of a family.

### Answer 2:

In [14]:
qsents = """
The monster is on the ice
Flowers are happy things
I have never seen the aurora borealis
He never knew the love of a family
""".split('\n')[1:-1]

In [15]:
QUEST_SENTS = pd.DataFrame({'sent_str':qsents})
QUEST_SENTS.index.name = 'sent_num'
QUEST_SENTS

,sent_str
sent_num,
0,The monster is on the ice
1,Flowers are happy things
2,I have never seen the aurora borealis
3,He never knew the love of a family


In [16]:
SENT_TOKEN = QUEST_SENTS.sent_str.str.split(expand=True).stack().to_frame('token_str')
SENT_TOKEN.index.names = ['sent_num', 'token_num']
SENT_TOKEN['term_str'] = SENT_TOKEN.token_str.str.replace(r'[\W_]+', '').str.lower()
SENT_TOKEN

token_str  term_str
sent_num token_num                    
0        0               The       the
         1           monster   monster
         2                is        is
         3                on        on
         4               the       the
         5               ice       ice
1        0           Flowers   flowers
         1               are       are
         2             happy     happy
         3            things    things
2        0                 I         i
         1              have      have
         2             never     never
         3              seen      seen
         4               the       the
         5            aurora    aurora
         6          borealis  borealis
3        0                He        he
         1             never     never
         2              knew      knew
         3               the       the
         4              love      love
         5                of        of
         6                 a         a
         7            family    family

In [17]:
SENT_NGRAMS = get_ngrams(SENT_TOKEN, n=2, sent_key='sent_num')

In [18]:
SENT_NGRAMS

w0        w1
sent_num token_num                    
0        0               <s>       the
         1               the   monster
         2           monster        is
         3                is        on
         4                on       the
         5               the       ice
         6               ice       <s>
         7               <s>       <s>
1        0               <s>   flowers
         1           flowers       are
         2               are     happy
         3             happy    things
         4            things       <s>
         5               <s>       <s>
2        0               <s>         i
         1                 i      have
         2              have     never
         3             never      seen
         4              seen       the
         5               the    aurora
         6            aurora  borealis
         7          borealis       <s>
         8               <s>       <s>
3        0               <s>        he
         1                he     never
         2             never      knew
         3              knew       the
         4               the      love
         5              love        of
         6                of         a
         7                 a    family
         8            family       <s>
         9               <s>      None

In [19]:
def test_model(model, test_ngrams):

    # Get the model level and info feature
    n = len(model.index.names) - 1 
    f = 'c' * bool(n) + 'i'        

    # Do the test by join and then split-apply-combine
    T = test_ngrams.join(model[f], on=model.index.names).fillna(model[f].max()).copy()
    
    R = T.groupby('sent_num')[f].agg(['sum','mean'])
    R['pp'] = np.exp2(R['mean'])
    
    return R

In [20]:
bgcounts = get_ngram_counts(bigrams)

In [21]:
RESULT = pd.concat(
    [test_model(bgcounts[i], SENT_NGRAMS.iloc[:,:i+1]) for i in range(len(bgcounts))],
    keys=[f"M{n}" for n in range(len(bgcounts))],
    axis=1
)

In [22]:
RESULT

M0                             M1                      
                 sum       mean       pp        sum      mean          pp
sent_num                                                                 
0         131.170965  16.396371  86258.0  45.124795  5.640599   49.887253
1          98.378224  16.396371  86258.0  56.461554  9.410259  680.409245
2         147.567336  16.396371  86258.0  64.225421  7.136158  140.668731
3         163.963706  16.396371  86258.0  77.805025  7.780503  219.869326

In [23]:
pd.concat([QUEST_SENTS, RESULT['M1']], axis=1).sort_values('pp').style.background_gradient()

,sent_str,sum,mean,pp
sent_num,,,,
0,The monster is on the ice,45.124795,5.640599,49.887253
2,I have never seen the aurora borealis,64.225421,7.136158,140.668731
3,He never knew the love of a family,77.805025,7.780503,219.869326
1,Flowers are happy things,56.461554,9.410259,680.409245


## Question 3:
Using the bigram model represented as a matrix, explore the relationship between bigram pairs using the following lists. Hint: use the .unstack() method on the feature n and then use .loc[] to select the first list from the index, and the second list from the columns.

1) ['he','she'] to select the indices.
2) ['said','heard'] to select the columns.

### Answer 3:

In [24]:
bgcounts[1].n.unstack()

w1,1,11th,12th,13th,17,1816,1817,18th,19,2,...,younger,youngest,youngster,your,yours,yourself,yourselves,youth,youthful,zeal
w0,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11th,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12th,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13th,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yourself,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
yourselves,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
youth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
bgcounts[1].n.unstack().loc[['he', 'she']].loc[['said', 'heard']]

KeyError: "None of [Index(['said', 'heard'], dtype='object', name='w0')] are in the [index]"

## Question 4:
Generate 20 sentences using the generate_text() function. Display the results.

### Answer 4

In [26]:
def generate_text(model, n=250):
    
    m1, m2, m3 = model
    start_word = m1.sample(weights='p').index[0][0]
    words = [start_word]
    
    for i in range(n):
        
        if len(words) == 1:
            next_word = m2.loc[start_word].sample(weights='p').index[0]
        
        elif len(words) > 1:

            # Get previous two words
            bg = tuple(words[-2:])
            
            # Try trigram model
            try:
                next_word = m3.loc[bg].sample(weights='cp').index[0]
            
            # If not in model, back off ...
            except KeyError:
                
                # Get the last word in the bigram
                ug = bg[1] 

                if ug == '<s>':
                    next_word = m1.sample(weights='p').index[0][0]
                else:
                    next_word = m2.loc[ug].sample(weights='cp').index[0]
            
        # Some words are returned as single item tuples
        if isinstance(next_word, tuple):
            next_word = next_word[0]
        
        words.append(next_word)
    
    text = ' '.join(words)
    lines = text.split('<s> <s>')
    for line in lines:
        print(line.strip().upper())

In [27]:
generate_text(tgcounts, n=400)

SUPPORTED ME IN DEJECTION
BUT I WAS A SHOW OF GRATITUDE AND WORSHIP IN HIS LETTERS AND ONLY FELT AS IF I SHOULD DESERVE THEIRS
WE RISE
I FOUND MY FEET
AND THE CHIVALROUS TRAIN WHO SHED THEIR BLOOD AND TO LOSE
THIS AROUSED THE STRANGER S ATTENTION
IT IS TRUE WE SHALL ASSUREDLY BE HAPPY
SOMETIMES HE HIMSELF WHO FEARED THAT HIS SUFFERINGS HAD DEPRIVED HIM OF UNDERSTANDING
THE CAVES OF THE MOUNTAINS OF OUR FUTURE PROSPECTS
SHE HAD AT FIRST BEEN SILENT AND APPEARS UNEASY WHEN ANY ONE BUT SAT MOTIONLESS BEWILDERED BY THE USE OF WHICH I MIGHT BY HIS CREW I FELT THEN THAT I SHOULD SCARCELY REGRET SWITZERLAND AND THE LIVELY CONVERSATION OF CLERVAL WAS THE PERIOD FIXED FOR THE DANGEROUS MYSTERIES OF CREATION
THEY KNOW OUR INFANTINE DISPOSITIONS WHICH HOWEVER WONDERFUL FORCES CONVICTION
OH THAT I CANNOT FIND WORDS TO YOU AND THAT HE WAS INQUISITIVE TO KNOW HIS VALUE AND LOSE HIM
I FOUND MYSELF AS ALWAYS HAVING BEEN THUS OCCUPIED FOR A MINUTE DOUBT MURDERED MY BROTHER
THE MISERABLE MONSTER WHOM I 

## Question 5
Compute the redundancy for each of the n-gram models using the MLE of the joint probability of each ngram type. In other words, for each model, just use the `.mle` feature as $p$ in computing $H = \sum{p(ng)log_2(1/p(ng))}$. Does $R$ increase, decrease, or remain the same as the choice of n-gram increases in length? Hint: Remember that $R = 1-\frac{H}{H_{max}}$, where $H$ is the actual entropy of the model and $H_{max}$ is its maximum entropy.

* If mle is not a feature in your models, just use p for the unigram model and compute p for the other two models by dividing n by the sum of n, i.e.

    ```python
    M[1]['p'] = M[1].n /  M[1].n.sum()
    M[2]['p'] = M[2].n /  M[2].n.sum()
    ```

* N is computed as the number of all possible combinations for each ngram. So, for the bigram model N is the number of unigrams (i.e. the vocabulary size plus the sentence boundary signs) squared, and for the trigram model the value is cubed, i.e.

    ```python
    N = len(M[0].index)**{i+1}
    ```

### Answer 5:


In [44]:
h_uni = sum(tgcounts[0].p*np.log2(1/tgcounts[0].p))
h_bi = sum((tgcounts[1].n/tgcounts[1].n.sum*())*np.log2(1/(tgcounts[1].n/tgcounts[1].n.sum*())))
h_tri = sum((tgcounts[2].n/tgcounts[2].n.sum*())*np.log2(1/(tgcounts[2].n/tgcounts[2].n.sum*())))

TypeError: unsupported operand type(s) for /: 'int' and 'method'